In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install langchain
!pip install -U langchain-community
#!pip install bitsandbytes
!pip install --upgrade trl
# !pip install transformers==4.35.0
!pip install accelerate
!pip install -i https://test.pypi.org/simple/ bitsandbytes
!pip install --upgrade bitsandbytes
!pip install --upgrade accelerate
!pip install peft==0.10.0
!pip install transformers==4.37.2
!pip install chromadb

!pip install pymisp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 86.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.2
    Uninstalling transformers-4.37.2:
      Successfully uninstalled transformers-4.37.2
Looking in indexes: https://test.pypi.org/simple/
  Using cached transformers-4.37.2-py3-none-any.whl.metadata (129 kB)
  Using cached tokenizers-0.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.37.2-py3-none-any.whl (8.4 MB)
Using cached tokenizers-0.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tok

#VECTORIZE WITH CHROMA

In [ ]:
import os
import chromadb
from pymisp import PyMISP
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document

VECTOR_DB_PATH = "/content/drive/MyDrive/NCKH/vectorDB/sqli"
MISP_URL = "https://b50f-103-111-244-37.ngrok-free.app"
MISP_KEY = "mf7fsZ3gVmSnPFckt7jYzmHQU8wEaEmsY4vtr5GY"
#INFERENCE_OUTPUT_PATH = "/content/drive/MyDrive/NCKH/Inference/sqli_misp.csv"
EVENT_ID = 3697

embedding = HuggingFaceEmbeddings(model_name="/content/drive/MyDrive/NCKH/all-MiniLM-L6-v2")
vector_db = Chroma(persist_directory=VECTOR_DB_PATH, embedding_function=embedding)

def get_sqli_payloads(event_id):
    misp = PyMISP(MISP_URL, MISP_KEY, False)
    event = misp.get_event(event_id)

    sqli_payloads = []
    if "Event" in event and "Attribute" in event["Event"]:
        for attr in event["Event"]["Attribute"]:
            sqli_payloads.append(attr["value"])

    return sqli_payloads

sqli_payloads = get_sqli_payloads(EVENT_ID)

if not sqli_payloads:
    print(f"Lỗi: Không tìm thấy payload sqli từ Event {EVENT_ID} trên MISP!")
    exit()

# Save payload to VectorDB if not already there
documents = [Document(page_content=payload.strip(), metadata={"type": "sqli"}) for payload in sqli_payloads]
vector_db.add_documents(documents)
vector_db.persist()
print(f"Đã lưu {len(documents)} payloads sqli từ Event {EVENT_ID} vào Vector Database tại {VECTOR_DB_PATH}!")

# Connect to ChromaDB
chroma_client = chromadb.PersistentClient(path=VECTOR_DB_PATH)
collection = chroma_client.get_or_create_collection(name="payloads")

# Check if data exists
if len(collection.get()["documents"]) == 0:
    ids = [str(i) for i in range(len(sqli_payloads))]
    collection.add(ids=ids, documents=[p.strip() for p in sqli_payloads])
    print(f"Đã lưu payloads sqli từ MISP vào ChromaDB với {len(ids)} entries!")

#GEN PAYLOAD AND TEST PAYLOAD WITH WAF

In [ ]:
import subprocess
import os

BASE_DIR = "/content/drive/MyDrive/NCKH"
INFERENCE_DIR = os.path.join(BASE_DIR, "Inference")
EVALUATION_WAF_DIR = os.path.join(BASE_DIR, "WAF")
VECTOR_DB_PATH = os.path.join(BASE_DIR, "vectorDB", "sqli")
PRETRAIN_MODEL = os.path.join(INFERENCE_DIR, "models", "pretrain_model_sql")
FINETUNED_MODEL = os.path.join(INFERENCE_DIR, "models", "fine_tune_model_sql_modsecurity")
RAW_OUTPUT_CSV = os.path.join(INFERENCE_DIR, "sqli_misp_1.csv")
GRAMMAR_PATH = os.path.join(INFERENCE_DIR, "grammar", "bnf-sql.txt")
TRANSFORMED_PAYLOAD_CSV = os.path.join(BASE_DIR, "Payload", "SQLI", "sqli_misp_payload_1.csv")
# TRANSFORMED_PAYLOAD_CSV = os.path.join(BASE_DIR, "Payload", "SQLI", "testSQL.csv")
WAF_SCORE_CSV = os.path.join(BASE_DIR, "score-gpt2-only.csv")


In [ ]:
def generate_payloads():
    print("Running inference to generate payloads...")
    cmd = [
        "python", os.path.join(INFERENCE_DIR, "inferenceRag.py"),
        "--lm_name", FINETUNED_MODEL,
        "--ref_lm_name", PRETRAIN_MODEL,
        "--total_nums", "10",
        "--txt_in_len", "20",
        "--txt_out_len", "100",
        "--savePath", RAW_OUTPUT_CSV,
        "--vector_db_path", VECTOR_DB_PATH
    ]
    subprocess.run(cmd, check=True)
    print("Payloads generated and saved to:", RAW_OUTPUT_CSV)

def transform_payloads():
    print("Transforming payloads using grammar...")
    cmd = [
        "python", os.path.join(INFERENCE_DIR, "transform.py"),
        "--grammar_path", GRAMMAR_PATH,
        "--data_path", RAW_OUTPUT_CSV,
        "--output_path", TRANSFORMED_PAYLOAD_CSV
    ]
    subprocess.run(cmd, check=True)
    print("Transformed payloads saved to:", TRANSFORMED_PAYLOAD_CSV)

def evaluate_payloads():
    print("Evaluating payloads against WAF...")
    cmd = [
        "python", os.path.join(EVALUATION_WAF_DIR, "modsecEvaluation.py"),
        "--server", "https://d50d-103-111-244-37.ngrok-free.app/DVWA/vulnerabilities/sqli/?id=",
        "--input", TRANSFORMED_PAYLOAD_CSV,
        "--output", WAF_SCORE_CSV
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    print(result.stdout)

In [ ]:
def prepare_payloads():
    try:
        generate_payloads()
        transform_payloads()
        print("Payload preparation completed successfully.")
    except subprocess.CalledProcessError as e:
        print("Payload preparation failed during subprocess execution:", e)
    except Exception as e:
        print("Payload preparation encountered an error:", e)

In [ ]:
def evaluate_pipeline():
    try:
        evaluate_payloads()
        print("Payload evaluation completed successfully.")
    except subprocess.CalledProcessError as e:
        print("Evaluation failed during subprocess execution:", e)
    except Exception as e:
        print("Evaluation encountered an error:", e)

In [ ]:
if __name__ == "__main__":
    prepare_payloads()
    evaluate_pipeline()

In [ ]:
# prepare_payloads()

In [ ]:
# evaluate_pipeline()

Evaluating payloads against WAF...
[+] Loaded 64 payloads from /content/drive/MyDrive/NCKH/Payload/SQLI/sqli_misp_payload_1.csv
[DEBUG] URL: https://d50d-103-111-244-37.ngrok-free.app/DVWA/vulnerabilities/sqli/?id=0%23&Submit=Submit
[DEBUG] Status: 200
[DEBUG] Content (first 300 chars):
<!DOCTYPE html>

<html lang="en-GB">

	<head>
		<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />

		<title>Vulnerability: SQL Injection :: Damn Vulnerable Web Application (DVWA)</title>

		<link rel="stylesheet" type="text/css" href="../../dvwa/css/main.css" />

		<link rel="ico
[DEBUG] URL: https://d50d-103-111-244-37.ngrok-free.app/DVWA/vulnerabilities/sqli/?id=0(15&Submit=Submit
[DEBUG] Status: 200
[DEBUG] Content (first 300 chars):
<!DOCTYPE html>

<html lang="en-GB">

	<head>
		<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />

		<title>Vulnerability: SQL Injection :: Damn Vulnerable Web Application (DVWA)</title>

		<link rel="stylesheet" type="text/css" hr